In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, log_loss
from sklearn.metrics import roc_curve, roc_auc_score, auc

* What is the negative and positive target?
* One classifier for each class?
* Should we use the nome_micro?

# Classifying NotRequested/Susceptible Vs Resistant

So in this aproach each row corresponds to a isolation request. Two classes are created: one for representing cases were no resistant to J01CA was detected.

In [2]:
# Load data
data_loaded = pd.read_csv("data2classificy_j01ca_n2.csv",sep=";")

# Drop columns that must not be used
columns2drop = ['ID_Isol','ID_Episodio','ID_Pedido','ID_Doente',
                'Resistant_concat_Fam','Resistant_Anti','Infeccoes','Requested_Fam',
                'DT_Admin','DT_Admin_temp','COLHEITA_DIFF','Dt_Colheita']
data = data_loaded.drop(columns2drop,axis=1)

# Check if there is missing data
data.isnull().sum()[data.isnull().sum()>0]


data.columns.tolist()

['Nome_Micro',
 'IDADE',
 'Genero',
 'Latitude',
 'Longitude',
 'Target_J01CA',
 'Diagnosticos_new',
 'Nome_Analises_new',
 'Produto_Analises_new',
 'Sintomas_new',
 'nr_past_visits',
 'nr_past_visits_7days',
 'nr_past_visits_15days',
 'nr_past_infections',
 'nr_past_infections_7days',
 'nr_past_infections_15days',
 'fever_7days',
 'fever_15days',
 'cough_7days',
 'cough_15days',
 'diarrhea_7days',
 'diarrhea_15days',
 'fatigue_7days',
 'fatigue_15days',
 'J01CA_past_resistance',
 'J01GB_past_resistance',
 'J01DH_past_resistance',
 'J01MA_past_resistance',
 'J01DD_past_resistance',
 'Past_Resistances',
 'Hour',
 'Weekday',
 'Monthday',
 'Month',
 'Year',
 'Temperature_3avg',
 'Temperature_3std',
 'Temperature_3min',
 'Humidity_3avg',
 'Humidity_3std',
 'Humidity_3min',
 'Temperature_5avg',
 'Temperature_5std',
 'Temperature_5min',
 'Humidity_5avg',
 'Humidity_5std',
 'Humidity_5min',
 'Temperature_7avg',
 'Temperature_7std',
 'Temperature_7min',
 'Humidity_7avg',
 'Humidity_7std',
 'Hu

So we see that are a bunch of variables that seem to have quite a amount of missing data. Looking to these variables we can divide them in 3 groups:
* Resistant_concat_Fam, Resistant_Anti, Diagnosticos_new, Nome_Analises_new, Produto_Analises_new, Sintomas_new, Infeccoes: When these variables are nan in fact means that missing is not random it means that no information was placed. The solution is just to replace these values by a value like 'Nenhum' to tell that no values were annotated by the clinical staff.
* Latitude/Longitude: No information was provided to these features solution (replace by a specific value and creat another feature indicating if these variable is missing)
* Weather features: These values are missing because we do not have data for the year 2013.

In [3]:
# Missing not at Random - empty values mean that a clinical observation was not written
data['Diagnosticos_new'].fillna("Nenhum",inplace=True)
data['Produto_Analises_new'].fillna("Nenhum",inplace=True)
data['Nome_Analises_new'].fillna("Nenhum",inplace=True)
data['Sintomas_new'].fillna("Nenhum",inplace=True)

# Fill the nan values with the localization of one of the clinicals
data['Longitude'].fillna(-9.1628837,inplace=True) 
data['Latitude'].fillna(38.748496,inplace=True)

# There are 4 cases with 
data['Nome_Micro'].fillna("Nenhum",inplace=True)

# select variables of weather:
weather_features = data.filter(regex='Temperature|Humidity').columns
# Input values based on month and monthday
for feat in weather_features:
    data[feat] = data_loaded.groupby(['Month', 'Monthday'])[feat].transform(lambda x: x.fillna(x.mean()))

# Check if the nan imoutation was sucessfull
data.isnull().sum()[data.isnull().sum()>0]

Past_Resistances    16182
dtype: int64

Features concerning hour,day of the week, are not continous numeric but are not categorical either. They have ordered. If monday is coded as 0 and sunday as 6 we have to find a way to make this 0-6 (monday-sunday) be more close that for example 0-5(monday saturday). A way to data is to make this variabvles cyclic using a cos or sin function 
http://blog.davidkaleko.com/feature-engineering-cyclical-features.html

One of the most common mistakes in data science is not properly dealing with cyclical features. Hours of the day, days of the week, months in a year, and wind direction are all examples of features that are cyclical. Many new machine learning engineers don’t think to convert these features into a representation that can preserve information such as hour 23 and hour 0 being close to each other and not far. Yup. I've tried to predict my fair share of quantities using cyclical features incorrectly. Christopher points out that proper handling of such features involves representing the cyclical features as (x,y) coordinates on a circle. In this blog post, I'll explore this feature engineering task and see if it really improves the predictive capability of a simple model. Now the magic happens. We map each cyclical variable onto a circle such that the lowest value for that variable appears right next to the largest value. We compute the x- and y- component of that point using sin and cos trigonometric functions. You remember your unit circle, right? Here's what it looks like for the "hours" variable. Zero (midnight) is on the right, and the hours increase counterclockwise around the circle. In this way, 23:59 is very close to 00:00, as it should be. Note that when we perform this transformation for the "month" variable, we also shift the values down by one such that it extends from 0 to 11, for convenience.

In [4]:
# Transform time variables cyclic with sin and cos fucntions
data['hour_sin'] = np.sin(data_loaded['Hour']*(2.*np.pi/24))
data['hour_cos'] = np.cos(data_loaded['Hour']*(2.*np.pi/24))
data['month_sin'] = np.sin((data_loaded['Month']-1)*(2.*np.pi/12))
data['month_cos'] = np.cos((data_loaded['Month']-1)*(2.*np.pi/12))
data['weekday_sin'] = np.sin(data_loaded['Weekday']*(2.*np.pi/7))
data['weekday_cos'] = np.cos(data_loaded['Weekday']*(2.*np.pi/7))
data['monthday_sin'] = np.sin(data_loaded['Monthday']*(2.*np.pi/30))
data['monthday_cos'] = np.cos(data_loaded['Monthday']*(2.*np.pi/30))

#data.drop(['Hour','Month','Weekday','Monthday'],axis=1,inplace=True)

We will need to dummerize the categorical variables becase other way random forest algorithm would be not capable of distinguish between integer and categorical variables.

In [5]:
def dumerize_bagofwords(dataset,feature,separator='--',drop_old=True):
    dummies = dataset[feature].str.get_dummies(sep=separator)
    for col_dummie_i in dummies.columns:
        dummies = dummies.rename(columns = {col_dummie_i:str(feature)+ '_' + str(col_dummie_i)})
        dummies = dummies.astype('uint8')
    data = pd.concat([dataset,dummies],axis=1)
    data = data.drop(feature,axis=1)
    return data

def dummerize_categorical(dataset,feature,drop_old=True):
    dummies = pd.get_dummies(dataset[feature],drop_first=True,prefix=feature)
    dummies = dummies.astype('uint8')
    data = pd.concat([dataset,dummies],axis=1)
    data = data.drop(feature,axis=1)
    return data

def remove_badfeatues(dataset,unique_thr=100):
    print(" ... Removing dummies that appear less than {} times:".format(unique_thr),end=" ")
    # Copy dataset to avoid probels
    dataset_out = dataset.copy()
    # Select categorical variables
    dataset_categorical = dataset_out.select_dtypes(include=['uint8'])
    # List to store the names of features to keep
    removidas = []
    # Ltst to store the names of features to drop
    mantidas = []
    # Iterate thourgh the uint columns
    for col in dataset_categorical.columns:
        if dataset_categorical[col].sum() < unique_thr:
            removidas.append(col)
            dataset_out.drop([col],axis=1,inplace=True)
        else:
            mantidas.append(col)
    # Print the number of columns to removed
    print("{} removed.".format(len(removidas)))
    return dataset_out,removidas

float_features = data.select_dtypes(include=['float64'])
numerical_features = data.filter(regex='nr_|IDADE|Latitude|Longitude|Humidity|Temperature').columns.tolist()
categorical_features = list(set(data.columns.tolist()) - set(numerical_features)-set(float_features))
categorical_features

data_new = data.copy()

for feat_cat in categorical_features:
    #if data_new[feat_cat].nunique()>2:
        print(feat_cat)
        if data_new[feat_cat].astype('object').str.contains('--').any():
            data_new = dumerize_bagofwords(data_new,feat_cat,separator='--',drop_old=True)
        else:
            data_new = dummerize_categorical(data_new,feat_cat,drop_old=True)       
        data_new,removidas = remove_badfeatues(data_new,unique_thr=5)
    

Diagnosticos_new
 ... Removing dummies that appear less than 5 times: 361 removed.
fatigue_15days
 ... Removing dummies that appear less than 5 times: 1 removed.
Month
 ... Removing dummies that appear less than 5 times: 0 removed.
J01CA_past_resistance
 ... Removing dummies that appear less than 5 times: 0 removed.
cough_15days
 ... Removing dummies that appear less than 5 times: 0 removed.
Nome_Analises_new
 ... Removing dummies that appear less than 5 times: 310 removed.
fatigue_7days
 ... Removing dummies that appear less than 5 times: 1 removed.
fever_7days
 ... Removing dummies that appear less than 5 times: 0 removed.
Target_J01CA
 ... Removing dummies that appear less than 5 times: 0 removed.
Past_Resistances
 ... Removing dummies that appear less than 5 times: 1056 removed.
Produto_Analises_new
 ... Removing dummies that appear less than 5 times: 27 removed.
Hour
 ... Removing dummies that appear less than 5 times: 0 removed.
Weekday
 ... Removing dummies that appear less than

Now we will separate the data and the targets and make a train and test set.

In [6]:
y = data_new['Target_J01CA_True']
X = data_new.drop('Target_J01CA_True',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
import scipy.stats as stats
col_list = []
pvalue_list = []
colunas  =  list(set(X.columns)-set(numerical_features)-set(float_features))
for col in colunas:
    #"if col not in continous_vars:
        #print("qui test for ", col)
        observed = pd.crosstab(X[col],y, margins = False)

        #print(observed)
        chi2, p, dof, ex = stats.chi2_contingency(observed= observed,correction = True)
        col_list.append(col)
        pvalue_list.append(p)

chi_test_pd = pd.DataFrame({'feature':col_list,'p_value':pvalue_list})


In [ ]:
print(chi_test_pd.sort_values(by='p_value', ascending=True).head(10))
print(chi_test_pd.sort_values(by='p_value', ascending=True).tail(10))
#chi_test_pd.to_csv("data2classificy_j01ca_n_chi_test_pd_dummies.csv",sep=";",encoding='utf-8',index=False)

g = sns.factorplot(x="Nome_Analises_new_T4  LIVRE  (FT4)", hue="Target_J01CA_True",data=data_new, kind="count",size=4, aspect=.7);
plt.show()

print(data_new["Nome_Analises_new_BIL   TOTAL "].value_counts())
print(data_new["Nome_Analises_new_GGT "].value_counts())
print(data_new["Produto_Analises_new_FEZES (MEIO DE TRANSPORTE)"].value_counts())
print(data_new["Nome_Analises_new_T4  LIVRE  (FT4)"].value_counts())

In [ ]:
feature_selection = True
if feature_selection:
    ## This line instantiates the model. 
    rf = RandomForestClassifier(n_estimators=50) 
    ## Fit the model on your training data.
    rf.fit(X_train, y_train) 
    ## And score it on your testing data.
    rf.score(X_test, y_test)
    # Build a dataframe with the feature importance sorted 
    feature_importances = pd.DataFrame(rf.feature_importances_,
                                       index = X_train.columns,
                                      columns=['importance']).sort_values('importance',ascending=False)
    # Subset the best 1000 features
    selected_features = feature_importances.index[0:1000]
    X_train = X_train[selected_features]
    X_test = X_test[selected_features]
    


## Train model -  Random Forest

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1,max_features='sqrt',n_estimators=10,oob_score=False)
optimize_parameters = True

if optimize_parameters:
    # The scorers can be either be one of the predefined metric strings or a scorer
    # callable, like the one returned by make_scorer
    scoring = {'AUC': 'roc_auc','Accuracy':'accuracy'}

    param_grid = {
        'n_estimators':[1,2,5,10,20,50,100],
        'max_features':['auto','sqrt','log2',0.5,0.8]
    }
    
    CV_rfc = GridSearchCV(estimator=rfc,param_grid=param_grid,cv=5,scoring=scoring,refit='Accuracy',return_train_score=True,verbose=2)
    CV_rfc.fit(X_train,y_train)
    
print(CV_rfc.best_params_)

In [ ]:
def plot_grid_search_traintime(cv_results,grid_param_1,grid_param_2,grid_param_3,name_param_1,name_param_2,name_param_3):
    # Plot Grid search scores
    _, ax = plt.subplots(2,2,figsize=(13,10))

    
     # Get Test Scores Mean and std for each grid search
    scores_mean = cv_results['mean_test_AUC']
    scores_mean = np.array(scores_mean).reshape(len(grid_param_2),len(grid_param_1))

    scores_sd = cv_results['std_test_AUC']
    scores_sd = np.array(scores_sd).reshape(len(grid_param_2),len(grid_param_1))


    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for idx, val in enumerate(grid_param_2):
        ax[0,0].plot(grid_param_1, scores_mean[idx,:], ':o',markersize=4, label= name_param_2 + ': ' + str(val))

    ax[0,0].set_title("Grid Search Scores", fontsize=20, fontweight='bold')
    ax[0,0].set_xlabel(name_param_1, fontsize=14)
    ax[0,0].set_ylabel('CV Average AUC', fontsize=14)
    ax[0,0].legend(loc="best", fontsize=12)
    ax[0,0].grid('on')
    
         # Get Test Scores Mean and std for each grid search
    scores_mean = cv_results['mean_test_Accuracy']
    scores_mean = np.array(scores_mean).reshape(len(grid_param_2),len(grid_param_1))

    scores_sd = cv_results['std_test_Accuracy']
    scores_sd = np.array(scores_sd).reshape(len(grid_param_2),len(grid_param_1))


    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for idx, val in enumerate(grid_param_2):
        ax[0,1].plot(grid_param_1, scores_mean[idx,:], ':o',markersize=4, label= name_param_2 + ': ' + str(val))

    ax[0,1].set_title("Grid Search Scores", fontsize=20, fontweight='bold')
    ax[0,1].set_xlabel(name_param_1, fontsize=14)
    ax[0,1].set_ylabel('CV Average Accuracy', fontsize=14)
    ax[0,1].legend(loc="best", fontsize=12)
    ax[0,1].grid('on')

    
         # Get Test Scores Mean and std for each grid search
#     scores_mean = cv_results['mean_test_Accuracy']
#     scores_mean = np.array(scores_mean).reshape(len(grid_param_3),len(grid_param_1))

#     scores_sd = cv_results['std_test_Accuracy']
#     scores_sd = np.array(scores_sd).reshape(len(grid_param_3),len(grid_param_1))


#     # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
#     for idx, val in enumerate(grid_param_3):
#         ax[1,0].plot(grid_param_1, scores_mean[idx,:], ':o',markersize=4, label= name_param_3 + ': ' + str(val))

#     ax[1,0].set_title("Grid Search Scores", fontsize=20, fontweight='bold')
#     ax[1,0].set_xlabel(name_param_1, fontsize=14)
#     ax[1,0].set_ylabel('CV Average Accuracy', fontsize=14)
#     ax[1,0].legend(loc="best", fontsize=12)
#     ax[1,0].grid('on')
    
    
    
    time_mean = cv_results['mean_fit_time']
    time_mean = np.array(time_mean).reshape(len(grid_param_2),len(grid_param_1))
    
    time_sd = cv_results['std_fit_time']
    time_sd = np.array(time_sd).reshape(len(grid_param_2),len(grid_param_1))
    
    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for idx, val in enumerate(grid_param_2):
        ax[1,1].plot(grid_param_1, time_mean[idx,:], ':o',markersize=4, label= name_param_2 + ': ' + str(val))

    ax[1,1].set_title("Grid Search Scores", fontsize=20, fontweight='bold')
    ax[1,1].set_xlabel(name_param_1, fontsize=14)
    ax[1,1].set_ylabel('CV Average Train Time', fontsize=14)
    ax[1,1].legend(loc="best", fontsize=12)
    ax[1,1].grid('on')
    plt.tight_layout()
    plt.show()
    
# Calling Method
plot_grid_search_traintime(CV_rfc.cv_results_, 
                           param_grid['n_estimators'], 
                           param_grid['max_features'], 
                           'N Estimators', 
                           'Max Features')


In [ ]:
# Optimized RF classifier
rfc_optimized = RandomForestClassifier(n_estimators=500, max_features=0.8, min_samples_leaf = 1,n_jobs=-1)

# Fit the model with training set
rfc_optimized.fit(X_train,y_train)



## Test model - Random Forest

In [ ]:
import seaborn as sns

def build_roc_cnf(y_true,y_score,y_predicted):
    fpr,tpr,thresholds = roc_curve(y_true,y_score[:,1])
    roc_auc = auc(fpr,tpr)
    
    plt.figure(figsize=(12,6))
    
    plt.subplot(1,2,1)
    plt.plot(fpr,tpr,color='darkorange',lw=2,label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0,1],[0,1],color='navy',lw=2,linestyle='--')
    plt.xlim([-0.01,1.0])
    plt.ylim([0.0,1.01])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc='lower right')
    
    plt.subplot(1,2,2)
    mat = confusion_matrix(y_true,y_predicted)
    sns.heatmap(mat.T, square = True, annot=True, fmt='d', cmap = 'Blues')
    plt.xlabel('true label')
    plt.ylabel('predicted label')
    plt.tight_layout()
    plt.show()


def plot_feature_importances(model, feature_names, autoscale=True, headroom=0.05, width=10, summarized_columns=None,topn=20,filename='feature_importance.png'):
    
    if autoscale:
        x_scale = model.feature_importances_.max() + headroom
    else:
        x_scale = 1

    feature_dict=dict(zip(feature_names, model.feature_importances_))

    plt.figure(figsize=(12,6))

    if summarized_columns:
        for col_name in summarized_columns:
            sum_value = sum(x for i, x in feature_dict.items() if col_name in i )
            keys_to_remove = [i for i in feature_dict.keys() if col_name in i ]
            for i in keys_to_remove:
                feature_dict.pop(i)
            feature_dict[col_name] = sum_value
    results = pd.Series(feature_dict, index=feature_dict.keys())
    results.sort_values(inplace=True)
    results = results[-topn:]
    results.plot(kind='barh', figsize=(width, len(results)/4), xlim=(0, x_scale))
    plt.title("Feature Relative Importance")
    plt.tick_params(axis='both', labelsize=9)
    plt.tight_layout()

    #plt.savefig(filename)

    
# Predict on testing set
y_predicted = rfc_optimized.predict(X_test)
y_predicted_scores = rfc_optimized.predict_proba(X_test)
target_names = ['J01CA Suscpetible/Not_Requested', 'J01CA Resistant']

#print("Confusion matrix: \n", confusion_matrix(y_test, y_predicted))
print("Classification report: \n", classification_report(y_test, y_predicted, target_names=target_names))
print("Classification accuracy: \n", accuracy_score(y_test, y_predicted))
print("Classification auc: \n", roc_auc_score(y_test, y_predicted_scores[:,1]))
print("Classification logloss: \n", log_loss(y_test,y_predicted_scores[:,1]))

build_roc_cnf(y_test,y_predicted_scores,y_predicted)

w = pd.DataFrame({'target':y_test,'probs':y_predicted_scores[:,1]})
plt.figure(figsize=(12,6))
sns.distplot(w.loc[w['target']==0,'probs'],bins=50,hist_kws={'range':(0,1),'rwidth':0.9, 'alpha':0.4},label=target_names[0])
sns.distplot(w.loc[w['target']==1,'probs'],bins=50,hist_kws={'range':(0,1),'rwidth':0.9, 'alpha':0.4},label=target_names[1])
plt.legend()
plt.xlim([0,1])
plt.show()

plot_feature_importances(model = rfc_optimized,feature_names = X_train.columns)
plt.show()

plot_feature_importances(model = rfc_optimized,feature_names = X_train.columns,summarized_columns = categorical_features)
plt.show()

## Classifying Suscpetible vs Resistant

In [ ]:
# Load data
data_loaded = pd.read_csv("data2classificy_j01ca_n.csv",sep=";")

# Subset rows to only pick rows where the j01ca class has been requested
data_loaded = data_loaded.filter(items = ['Requested_Fam'],like='J01CA')

# Drop columns that must not be used
columns2drop = ['ID_Isol','ID_Episodio','ID_Pedido','ID_Doente',
                'Resistant_concat_Fam','Resistant_Anti','Infeccoes',
                'DT_Admin','COLHEITA_DIFF','Dt_Colheita']
data = data_loaded.drop(columns2drop,axis=1)

# Check if there is missing data
data.isnull().sum()[data.isnull().sum()>0]